Since my major question focuses on gathering data from the BRICS nations, one of the initial problems faced was trying to find reliable sources to same basal columns for the five countries of Brazil, India, China, South Africa and Russia. 

Whilst finding the data for similar columns, one of the issues I had faced for, for instance, if China had information about Education, which was also one of the parameters I wanted to explore, South Africa's education did not have cut offs in data collection for primary, secondary and tertiary schools. If Adjusted National was calculated for Brazil, India's Net National Income wasn't properly adjusted. Hence, usage of API was indeed the best step forward as it helped calculated all the data points on an equal scale. 


# Data Source 

Since the data is that is majorly required is numerical data. I am retrieving data through API's and by downloading data from World Bank.

## Quantitative Data

The data required for this project is a culmunation of macroeconomic and fiscal indicators of all the five emerging economics for which individual data has to be retrived using various sources.

### Python API - FRED API

The Python API used for this project is the FRED API. The FRED-API is a web service that allows developers to create programmes and construct apps that retrieve statistical information about the economy from the FRED and ALFRED websites hosted by the Federal Reserve Bank of St. Louis' Economic Research Division using Python. 

Since we are using data of the 5 countries, it can be interesting to compare it with a hegemonic and the most powerful country in the world, The United States of America so using the FRED-API, you can clone few macroeconomic indicators from here. 

The FRED API has been called and the series IDs have been added to the data.

In [1]:
from fredapi.fred import Fred
import pandas as pd
fred = Fred(api_key='10fbe66f8f62ad7f44097cca867bf01f')

ImportError: cannot import name 'Fred' from partially initialized module 'fredapi' (most likely due to a circular import) (/Users/sheebamoghal/Desktop/DSAN5000/dsan-5000-project-msheeba4848/dsan-website/5000-website/codes/fredapi.py)

Note: you may need to restart the kernel to use updated packages.


## Textual Data

### Python API